In [11]:
"""
Exercise 01: Model Persistence - Starter Code

Master different model saving formats.

Prerequisites:
- Reading: 01-saving-loading-models.md
- Demo: demo_01_save_load_models.py (KEY REFERENCE FOR FORMATS)
"""

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import numpy as np
import os
import shutil

# Setup
SAVE_DIR = 'saved_models'
if os.path.exists(SAVE_DIR):
    shutil.rmtree(SAVE_DIR)
os.makedirs(SAVE_DIR)

In [12]:

# ============================================================================
# SETUP (PROVIDED)
# ============================================================================

def create_and_train_model():
    """Create and briefly train a model for testing."""
    (x_train, y_train), (x_test, y_test) = keras.datasets.mnist.load_data()
    x_train = x_train[:5000].reshape(-1, 784).astype('float32') / 255.0
    x_test = x_test.reshape(-1, 784).astype('float32') / 255.0
    y_train = y_train[:5000]
    
    model = keras.Sequential([
        layers.Dense(128, activation='relu', input_shape=(784,)),
        layers.Dropout(0.2),
        layers.Dense(64, activation='relu'),
        layers.Dense(10, activation='softmax')
    ], name='mnist_classifier')
    
    model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
    model.fit(x_train, y_train, epochs=5, batch_size=128, validation_split=0.2, verbose=1)
    
    return model, (x_test, y_test)


In [13]:

# ============================================================================
# TASK 1.1: Save Complete Models
# ============================================================================

def get_dir_size(path):
    """Get total size of a directory in bytes."""
    total = 0
    for dirpath, _, filenames in os.walk(path):
        for f in filenames:
            total += os.path.getsize(os.path.join(dirpath, f))
    return total

def format_size(size_bytes):
    """Format bytes to human readable string."""
    for unit in ['B', 'KB', 'MB']:
        if size_bytes < 1024:
            return f"{size_bytes:.2f} {unit}"
        size_bytes /= 1024
    return f"{size_bytes:.2f} GB"

def save_complete_model(model):
    """
    Save model in different formats and compare.
    """
    print("Task 1.1: Saving Complete Models\n")
    
    # 1. Save as .keras (recommended for Keras 3)
    keras_path = os.path.join(SAVE_DIR, 'model.keras')
    model.save(keras_path)
    keras_size = os.path.getsize(keras_path)
    print(f"✓ .keras format saved: {keras_path}")
    print(f"  Size: {format_size(keras_size)}")
    print(f"  Contains: architecture + weights + optimizer state + training config\n")
    
    # 2. Save as .h5 (legacy HDF5 format)
    h5_path = os.path.join(SAVE_DIR, 'model.h5')
    model.save(h5_path)
    h5_size = os.path.getsize(h5_path)
    print(f"✓ .h5 format saved: {h5_path}")
    print(f"  Size: {format_size(h5_size)}")
    print(f"  Contains: architecture + weights + optimizer state + training config\n")
    
    # 3. Save as SavedModel directory (for TF Serving)
    savedmodel_path = os.path.join(SAVE_DIR, 'savedmodel')
    model.export(savedmodel_path)
    savedmodel_size = get_dir_size(savedmodel_path)
    print(f"✓ SavedModel format saved: {savedmodel_path}")
    print(f"  Size: {format_size(savedmodel_size)}")
    print(f"  Contains: TensorFlow graph + variables (optimized for serving)\n")
    
    print("=" * 50)
    print("Format Comparison:")
    print(f"  .keras:     {format_size(keras_size)}")
    print(f"  .h5:        {format_size(h5_size)}")
    print(f"  SavedModel: {format_size(savedmodel_size)}")

In [14]:

# ============================================================================
# TASK 1.2: Save Weights Only
# ============================================================================

def save_weights_only(model):
    """
    Save just the model weights (no architecture).
    """
    print("Task 1.2: Saving Weights Only\n")
    
    weights_path = os.path.join(SAVE_DIR, 'model.weights.h5')
    model.save_weights(weights_path)
    weights_size = os.path.getsize(weights_path)
    
    print(f"Weights saved: {weights_path}")
    print(f"  Size: {format_size(weights_size)}")
    print(f"  Contains: weights only (no architecture/optimizer)")
    print(f"\n  Use case: version-controlled code, transfer learning, smaller files")

In [15]:

# ============================================================================
# TASK 1.3: Load and Verify
# ============================================================================

def load_and_verify(model, test_data):
    """
    Load each saved model and verify it matches original.
    """
    print("Task 1.3: Load and Verify\n")
    
    x_test, y_test = test_data
    x_subset = x_test[:100]
    
    # 1. Evaluate original model
    orig_loss, orig_acc = model.evaluate(x_test, y_test, verbose=0)
    orig_preds = model.predict(x_subset, verbose=0)
    print(f"Original Model - Loss: {orig_loss:.4f}, Accuracy: {orig_acc:.4f}\n")
    
    # 2. Load and verify .keras format
    keras_model = keras.models.load_model(os.path.join(SAVE_DIR, 'model.keras'))
    keras_loss, keras_acc = keras_model.evaluate(x_test, y_test, verbose=0)
    keras_preds = keras_model.predict(x_subset, verbose=0)
    keras_match = np.allclose(orig_preds, keras_preds)
    print(f".keras Model  - Loss: {keras_loss:.4f}, Accuracy: {keras_acc:.4f}, Predictions Match: {keras_match}")
    
    # 3. Load and verify .h5 format
    h5_model = keras.models.load_model(os.path.join(SAVE_DIR, 'model.h5'))
    h5_loss, h5_acc = h5_model.evaluate(x_test, y_test, verbose=0)
    h5_preds = h5_model.predict(x_subset, verbose=0)
    h5_match = np.allclose(orig_preds, h5_preds)
    print(f".h5 Model     - Loss: {h5_loss:.4f}, Accuracy: {h5_acc:.4f}, Predictions Match: {h5_match}")
    
    # 4. Load and verify SavedModel (via TFSMLayer)
    savedmodel_layer = keras.layers.TFSMLayer(os.path.join(SAVE_DIR, 'savedmodel'), call_endpoint='serve')
    sm_preds = savedmodel_layer(x_subset).numpy()
    sm_match = np.allclose(orig_preds, sm_preds)
    print(f"SavedModel    - Predictions Match: {sm_match} (TFSMLayer - no evaluate)")
    
    # 5. Load weights into fresh model
    fresh_model = keras.Sequential([
        layers.Dense(128, activation='relu', input_shape=(784,)),
        layers.Dropout(0.2),
        layers.Dense(64, activation='relu'),
        layers.Dense(10, activation='softmax')
    ])
    fresh_model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
    fresh_model.load_weights(os.path.join(SAVE_DIR, 'model.weights.h5'))
    weights_loss, weights_acc = fresh_model.evaluate(x_test, y_test, verbose=0)
    weights_preds = fresh_model.predict(x_subset, verbose=0)
    weights_match = np.allclose(orig_preds, weights_preds)
    print(f"Weights Model - Loss: {weights_loss:.4f}, Accuracy: {weights_acc:.4f}, Predictions Match: {weights_match}")
    
    print("\n✓ All formats verified successfully!" if all([keras_match, h5_match, sm_match, weights_match]) else "\n✗ Some formats did not match!")


In [16]:

# ============================================================================
# TASK 1.4: Format Comparison Report
# ============================================================================

def create_comparison_report():
    """Create report.md comparing formats."""
    keras_size = format_size(os.path.getsize(os.path.join(SAVE_DIR, 'model.keras')))
    h5_size = format_size(os.path.getsize(os.path.join(SAVE_DIR, 'model.h5')))
    savedmodel_size = format_size(get_dir_size(os.path.join(SAVE_DIR, 'savedmodel')))
    weights_size = format_size(os.path.getsize(os.path.join(SAVE_DIR, 'model.weights.h5')))
    
    report = f"""# Model Format Comparison Report

| Format     | Size          | Contains              | Use Case           |
|------------|---------------|----------------------|-------------------|
| .keras     | {keras_size:13} | arch + weights + opt | Default choice    |
| .h5        | {h5_size:13} | arch + weights + opt | Legacy support    |
| SavedModel | {savedmodel_size:13} | TF graph + vars      | TF Serving        |
| .weights   | {weights_size:13} | weights only         | Transfer learning |
"""
    
    with open('report.md', 'w') as f:
        f.write(report)
    
    print("✓ report.md created")
    print(report)

In [17]:

# ============================================================================
# MAIN
# ============================================================================

if __name__ == "__main__":
    print("=" * 60)
    print("Exercise 01: Model Persistence")
    print("=" * 60)
    
    # Uncomment as you complete:
    model, test_data = create_and_train_model()
    save_complete_model(model)
    save_weights_only(model)
    load_and_verify(model, test_data)
    create_comparison_report()


Exercise 01: Model Persistence
Epoch 1/5
32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.5675 - loss: 1.4966 - val_accuracy: 0.7930 - val_loss: 0.7129
Epoch 2/5
32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.8278 - loss: 0.5932 - val_accuracy: 0.8810 - val_loss: 0.4366
Epoch 3/5
32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.8848 - loss: 0.4016 - val_accuracy: 0.8970 - val_loss: 0.3651
Epoch 4/5
32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.9060 - loss: 0.3207 - val_accuracy: 0.9120 - val_loss: 0.3332
Epoch 5/5
32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.9258 - loss: 0.2739 - val_accuracy: 0.9150 - val_loss: 0.2965


Task 1.1: Saving Complete Models

✓ .keras format saved: saved_models/model.keras
  Size: 1.28 MB
  Contains: architecture + weights + optimizer state + training config

✓ .h5 format saved: saved_models/model.h5
  Size: 1.28 MB
  Contains: architecture + weights + optimizer state + training config

INFO:tensorflow:Assets written to: saved_models/savedmodel/assets


INFO:tensorflow:Assets written to: saved_models/savedmodel/assets


Saved artifact at 'saved_models/savedmodel'. The following endpoints are available:

* Endpoint 'serve'
  args_0 (POSITIONAL_ONLY): TensorSpec(shape=(None, 784), dtype=tf.float32, name='keras_tensor_70')
Output Type:
  TensorSpec(shape=(None, 10), dtype=tf.float32, name=None)
Captures:
  5603862544: TensorSpec(shape=(), dtype=tf.resource, name=None)
  5603864464: TensorSpec(shape=(), dtype=tf.resource, name=None)
  5603865424: TensorSpec(shape=(), dtype=tf.resource, name=None)
  5603864656: TensorSpec(shape=(), dtype=tf.resource, name=None)
  5603865808: TensorSpec(shape=(), dtype=tf.resource, name=None)
  5603865616: TensorSpec(shape=(), dtype=tf.resource, name=None)
✓ SavedModel format saved: saved_models/savedmodel
  Size: 904.32 KB
  Contains: TensorFlow graph + variables (optimized for serving)

Format Comparison:
  .keras:     1.28 MB
  .h5:        1.28 MB
  SavedModel: 904.32 KB
Task 1.2: Saving Weights Only

Weights saved: saved_models/model.weights.h5
  Size: 1.28 MB
  Contain

.keras Model  - Loss: 0.3090, Accuracy: 0.9114, Predictions Match: True
.h5 Model     - Loss: 0.3090, Accuracy: 0.9114, Predictions Match: True
SavedModel    - Predictions Match: True (TFSMLayer - no evaluate)


/Users/kyle-anthonyhay/Documents/CODE/Revature-Training/Ai-Engineering/December/venv/lib/python3.12/site-packages/keras/src/layers/core/dense.py:106: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
/Users/kyle-anthonyhay/Documents/CODE/Revature-Training/Ai-Engineering/December/venv/lib/python3.12/site-packages/keras/src/saving/saving_lib.py:797: UserWarning: Skipping variable loading for optimizer 'adam', because it has 2 variables whereas the saved optimizer has 14 variables. 
  saveable.load_own_variables(weights_store.get(inner_path))


Weights Model - Loss: 0.3090, Accuracy: 0.9114, Predictions Match: True

✓ All formats verified successfully!
✓ report.md created
# Model Format Comparison Report

| Format     | Size          | Contains              | Use Case           |
|------------|---------------|----------------------|-------------------|
| .keras     | 1.28 MB       | arch + weights + opt | Default choice    |
| .h5        | 1.28 MB       | arch + weights + opt | Legacy support    |
| SavedModel | 904.32 KB     | TF graph + vars      | TF Serving        |
| .weights   | 1.28 MB       | weights only         | Transfer learning |

